In [1]:
#### label 정보
#### (배경 - 0 / 몸통 - 1 / 오른손 - 2 / 왼손 - 3 / 왼발 - 4 / 오른발 - 5 / 오른쪽 허벅지 - 6 / 왼쪽 허벅지 - 7 / 오른쪽 종아리 - 8 / 왼쪽 종아리 - 9 / 왼쪽 팔 - 10 / 
#### 오른쪽 팔 - 11 / 왼쪽 전완 - 12 / 오른쪽 전완 - 13 / 머리 - 14)
#### 해당 값이 RGB 값임 ex) 배경 0,0,0
### mask 확장자 png input image 확장자 jpg 해깔리지 말것

In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)
import mmcv
import os.path as osp

1.9.0 True
0.18.0


In [3]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
## ver2용 추가
## 기존 mmsegmentation에 없거나 기능수정한 것 따로 data pipeline set
from custom_data_pipeline import CutOut,RandomAffine,CustomRandomFlip

In [4]:
classes = ('bg', 'body', 'right_hand', 'left_hand', 'left_leg', 'right_reg', 'right_thigh', 'left_thigh','right_calf','left_calf'
           ,'left_arm','right_arm','left_forearm','right_forearm','head')
## ver2용 추가 
flip_pair=[[2,3],[4,5],[6,7],[8,9],[10,11],[12,13]]  ##flip pair

In [5]:
palette = [[0,0,0], [1,1,1], [2,2,2], [3,3,3],[4,4,4], [5,5,5], [6,6,6], [7,7,7], [8,8,8], [9,9,9], [10,10,10], [11,11,11], [12,12,12], [13,13,13], [14,14,14]]

In [6]:
from mmcv import Config

In [7]:
from mmseg.apis import set_random_seed
cfg = Config.fromfile('mmsegmentation/configs/swin/upernet_swin_small_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K.py')

In [8]:
cfg.dataset_type = 'body_seg'
cfg.data_root = '/root'


# Since we use ony one GPU, BN is used instead of SyncBN
## encoding 부분은 layer normalization 
## encoding 이후 ecoder part 부터는 batchnormalization
cfg.norm_cfg = dict(type='LN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.auxiliary_head.norm_cfg = dict(type='BN', requires_grad=True)


# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 15
cfg.model.auxiliary_head.num_classes = 15

cfg.data.samples_per_gpu = 24
cfg.data.workers_per_gpu = 2

cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (224, 224)

# 기존 
# cfg.train_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(type='LoadAnnotations'),
#     dict(type='Resize', img_scale=(256, 256), ratio_range=(0.5, 2.0)),
#     dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
#     dict(type='RandomFlip', flip_ratio=0.5),
#     dict(type='PhotoMetricDistortion'),
#     dict(type='Normalize', **cfg.img_norm_cfg),
#     dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
#     dict(type='DefaultFormatBundle'),
#     dict(type='Collect', keys=['img', 'gt_semantic_seg']),
# ]

### 수정 version 2
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(256, 256), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.0),   # flip시 pair부위가 뒤집어져야 하기 때문에 custom flip 적용 
    dict(type='PhotoMetricDistortion'),
    ##### ver 2용 수정 #########
    dict(type='CustomRandomFlip', prob=0.5, flip_pair=flip_pair), #기존 flip과 동일하게 horizontal flip 하면서 pair를 맞춰주는 거만 추가함 
    dict(type='CutOut', n_holes=(0,4),cutout_shape=[(16,16),(32,32),(48,48)]),   ## 추가 (Cutout이 아닌 Random erase 형태로 적용함)
    #dict(type='RandomAffine'),              ## 추가 (affine transform은 조금 강하게 줬을 때 학습 경향이 너무 안좋아져서 default로 사용하는 것으로..)
                                            ## affine transform은 아주 약하게 input에만 먹이고 label은 적용 안하는 것으로..(라벨까지 적용시 학습경향이 안좋음)
    ##### ver 2용 수정 #########
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

## test시간 아껴야 되서 TTA나 후처리는 실제 inference시에만 적용 
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(224, 224),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = 'train2014'
cfg.data.train.ann_dir = 'train_mask'
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = 'splits/train.txt'

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = 'val2014'
cfg.data.val.ann_dir = 'val_mask'
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = 'splits/val.txt'


cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = 'val2014'
cfg.data.test.ann_dir = 'val_mask'
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = 'splits/val.txt'

## 강한 augmentation 주기 이전에 best validation iou score 기반 weight
# checkpoint_file = 'pretrained/upernet_swin_small_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K.pth'
checkpoint_file = 'pretrained/iter_173328.pt'
cfg.load_from = checkpoint_file

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/train_small_ver2'

# 1epoch 대략 43000
cfg.runner.max_iters = 1084*200
cfg.log_config.interval = 542
cfg.evaluation.interval = 1084
cfg.checkpoint_config.interval = 1084

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


cfg.optimizer = dict(
    type='AdamW',
    lr=0.0001,
    betas=(0.9, 0.999),
    weight_decay=0.,
    paramwise_cfg=dict(
        custom_keys={
            'absolute_pos_embed': dict(decay_mult=0.),
            'relative_position_bias_table': dict(decay_mult=0.),
            'norm': dict(decay_mult=0.)
        }))


print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='LN', requires_grad=True)
backbone_norm_cfg = dict(type='LN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained='pretrain/swin_small_patch4_window7_224.pth',
    backbone=dict(
        type='SwinTransformer',
        pretrain_img_size=224,
        embed_dims=96,
        patch_size=4,
        window_size=7,
        mlp_ratio=4,
        depths=[2, 2, 18, 2],
        num_heads=[3, 6, 12, 24],
        strides=(4, 2, 2, 2),
        out_indices=(0, 1, 2, 3),
        qkv_bias=True,
        qk_scale=None,
        patch_norm=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.3,
        use_abs_pos_embed=False,
        act_cfg=dict(type='GELU'),
        norm_cfg=dict(type='LN', requires_grad=True)),
    decode_head=dict(
        type='UPerHead',
        in_channels=[96, 192, 384, 768],
        in_index=[0, 1, 2, 3],
        pool_scales=(1, 2, 3, 6),
        channels=512,
        dropout_ratio=0.1,
        num_c

In [9]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

@DATASETS.register_module()
class body_seg(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None


In [10]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector

# config_file = 'mmsegmentation/configs/swin/upernet_swin_base_patch4_window12_512x512_160k_ade20k_pretrain_384x384_22K.py'
# model = init_segmentor(config_file, checkpoint_file, device='cuda:0')
model = build_segmentor(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

#학습시켰던 weight만 load 해줌
weights=torch.load('pretrained/iter_173328.pt')
model.load_state_dict(weights)


/opt/conda/lib/python3.7/site-packages/mmcv/utils/misc.py:334: UserWarning: "flip_ratio" is deprecated in `RandomFlip.__init__`, please use "prob" instead
  f'"{src_arg_name}" is deprecated in '
2021-11-16 05:11:30,381 - mmseg - INFO - Loaded 26437 images
/opt/conda/lib/python3.7/site-packages/mmseg/models/backbones/swin.py:553: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '


<All keys matched successfully>

In [ ]:
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

2021-11-16 05:11:34,239 - mmseg - INFO - Loaded 1908 images
2021-11-16 05:11:34,240 - mmseg - INFO - load checkpoint from pretrained/iter_173328.pt
2021-11-16 05:11:34,240 - mmseg - INFO - Use load_from_local loader
2021-11-16 05:11:34,461 - mmseg - INFO - Start running, host: root@67a5787c91ba, work_dir: /root/work_dirs/train_small_ver2
2021-11-16 05:11:34,462 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) I

[                                                  ] 0/1908, elapsed: 0s, ETA:

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1908/1908, 12.6 task/s, elapsed: 151s, ETA:     0s

2021-11-16 05:31:29,662 - mmseg - INFO - per class results:
2021-11-16 05:31:29,664 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      | 95.45 | 98.49 |
|      body     | 73.31 | 79.91 |
|   right_hand  | 14.47 | 24.65 |
|   left_hand   | 11.59 | 21.51 |
|    left_leg   |  7.1  | 11.32 |
|   right_reg   |  6.38 |  10.6 |
|  right_thigh  |  8.14 | 14.35 |
|   left_thigh  |  8.98 | 17.13 |
|   right_calf  |  8.26 | 15.98 |
|   left_calf   |  7.9  | 14.65 |
|    left_arm   | 11.31 |  21.4 |
|   right_arm   |  13.1 | 20.98 |
|  left_forearm |  9.98 | 19.29 |
| right_forearm |  11.4 | 20.19 |
|      head     | 66.25 | 73.76 |
+---------------+-------+-------+
2021-11-16 05:31:29,665 - mmseg - INFO - Summary:
2021-11-16 05:31:29,665 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 87.73 | 23.57 | 30.95 |
+-------+-------+-------+
2021-11-16 05:31:29,670 - mmse

...

2021-11-16 05:51:24,829 - mmseg - INFO - per class results:
2021-11-16 05:51:24,830 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      | 95.23 | 98.58 |
|      body     | 71.89 | 78.26 |
|   right_hand  | 14.68 | 24.97 |
|   left_hand   | 12.47 | 23.54 |
|    left_leg   |  9.01 | 14.46 |
|   right_reg   |  7.03 | 11.62 |
|  right_thigh  |  9.17 | 15.87 |
|   left_thigh  | 10.77 | 20.47 |
|   right_calf  |  8.72 | 15.64 |
|   left_calf   | 10.29 |  18.5 |
|    left_arm   | 12.46 | 22.98 |
|   right_arm   | 13.65 | 21.57 |
|  left_forearm |  10.9 | 20.89 |
| right_forearm | 11.92 | 20.66 |
|      head     | 65.08 |  72.2 |
+---------------+-------+-------+
2021-11-16 05:51:24,831 - mmseg - INFO - Summary:
2021-11-16 05:51:24,831 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 87.84 | 24.22 | 32.01 |
+-------+-------+-------+
2021-11-16 05:51:24,843 - mmse

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1908/1908, 12.5 task/s, elapsed: 153s, ETA:     0s

2021-11-16 06:11:21,595 - mmseg - INFO - per class results:
2021-11-16 06:11:21,596 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      | 95.22 | 98.56 |
|      body     | 71.71 | 77.57 |
|   right_hand  | 14.39 | 24.88 |
|   left_hand   | 11.84 | 22.27 |
|    left_leg   |  9.31 |  15.3 |
|   right_reg   |  5.29 |  8.46 |
|  right_thigh  |  7.94 | 13.81 |
|   left_thigh  |  9.58 |  18.5 |
|   right_calf  |  7.71 | 14.12 |
|   left_calf   |  9.56 | 17.55 |
|    left_arm   | 11.67 | 21.82 |
|   right_arm   | 13.85 | 22.85 |
|  left_forearm |  10.0 | 18.54 |
| right_forearm | 12.07 | 21.27 |
|      head     | 65.25 | 72.66 |
+---------------+-------+-------+
2021-11-16 06:11:21,597 - mmseg - INFO - Summary:
2021-11-16 06:11:21,597 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 87.66 | 23.69 | 31.21 |
+-------+-------+-------+
2021-11-16 06:11:21,600 - mmse

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1908/1908, 12.6 task/s, elapsed: 152s, ETA:     0s

2021-11-16 06:31:17,547 - mmseg - INFO - per class results:
2021-11-16 06:31:17,549 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      | 95.22 |  98.6 |
|      body     |  72.3 | 78.84 |
|   right_hand  | 14.21 | 23.77 |
|   left_hand   |  12.4 | 23.21 |
|    left_leg   |  9.23 | 15.32 |
|   right_reg   |  4.47 |  7.16 |
|  right_thigh  |  7.66 | 13.23 |
|   left_thigh  |  9.95 |  19.4 |
|   right_calf  |  5.93 | 10.53 |
|   left_calf   |  9.14 | 16.79 |
|    left_arm   | 12.18 | 22.93 |
|   right_arm   | 13.74 | 21.99 |
|  left_forearm | 10.72 | 19.45 |
| right_forearm | 11.82 | 20.31 |
|      head     | 65.27 | 72.29 |
+---------------+-------+-------+
2021-11-16 06:31:17,549 - mmseg - INFO - Summary:
2021-11-16 06:31:17,550 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 87.74 | 23.62 | 30.92 |
+-------+-------+-------+
2021-11-16 06:31:17,558 - mmse